In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/954471/954471/Train/M/121368482_3314461121964215_543249990768154260_o.jpg
/kaggle/input/954471/954471/Train/M/121338993_3559428640790674_3898861408853482856_o.jpg
/kaggle/input/954471/954471/Train/M/121543339_3481304788558895_349846518547631903_n.jpg
/kaggle/input/954471/954471/Train/M/121335583_2114014788729050_910466790392877863_n.jpg
/kaggle/input/954471/954471/Train/M/121478814_3277574072359599_3175620794088564281_n.jpg
/kaggle/input/954471/954471/Train/M/121381660_3348014425282334_9154920501988623614_o.jpg
/kaggle/input/954471/954471/Train/M/121472960_3362092607235628_5170860563284396140_o.jpg
/kaggle/input/954471/954471/Train/M/121222913_2888173951406016_2943121663743613328_n.jpg
/kaggle/input/954471/954471/Train/F/121256709_972711336544125_7929623606048122247_o.jpg
/kaggle/input/954471/954471/Train/F/121597221_972711616544097_204099069773999800_o.jpg
/kaggle/input/954471/954471/Train/F/121282643_1739796282838922_3924762867765759060_n.jpg
/kaggle/input/954471/954471

In [2]:
test_dir="/kaggle/input/954471/954471/Test"
train_dir="/kaggle/input/954471/954471/Train"

train_dir_F = train_dir + '/F'
train_dir_M = train_dir + '/M'
test_dir_F = test_dir + '/F'
test_dir_M = test_dir + '/M'

In [3]:
print('number of F training images - ',len(os.listdir(train_dir_F)))
print('number of M training images - ',len(os.listdir(train_dir_M)))
print('number of F testing images - ',len(os.listdir(train_dir_F)))
print('number of M testing images - ',len(os.listdir(train_dir_M)))

number of F training images -  8
number of M training images -  8
number of F testing images -  8
number of M testing images -  8


In [16]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        rescale = 1.0/255.0,   # Intensity Normalized
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,   # randomly flip images
        validation_split= 0.2)

batch_size = 2
img_height = 64 
img_width  = 64 
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

validation_generator = data_generator.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

Found 14 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


In [17]:
train_generator.class_indices

{'F': 0, 'M': 1}

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
# preparing the layers in the Convolutional Deep Neural Network
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = train_generator.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
# model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
fitted_model = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 20)

Epoch 1/20
7/7 [==============================] - 0s 48ms/step - loss: 0.8075 - accuracy: 0.4286 - val_loss: 0.6909 - val_accuracy: 0.5000
Epoch 2/20
7/7 [==============================] - 0s 19ms/step - loss: 0.6813 - accuracy: 0.5000 - val_loss: 0.6914 - val_accuracy: 1.0000
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 0.7206 - accuracy: 0.3571 - val_loss: 0.6892 - val_accuracy: 0.5000
Epoch 4/20
7/7 [==============================] - 0s 20ms/step - loss: 0.7247 - accuracy: 0.4286 - val_loss: 0.6875 - val_accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 0s 19ms/step - loss: 0.7025 - accuracy: 0.5714 - val_loss: 0.6859 - val_accuracy: 0.5000
Epoch 6/20
7/7 [==============================] - 0s 19ms/step - loss: 0.6658 - accuracy: 0.6429 - val_loss: 0.6909 - val_accuracy: 0.5000
Epoch 7/20
7/7 [==============================] - 0s 19ms/step - loss: 0.6840 - accuracy: 0.5000 - val_loss: 0.6836 - val_accuracy: 1.0000
Epoch 8/20
7/7 [===========

In [20]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/F/121297902_1739796202838930_2202079621057532442_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[0.]]


In [21]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/F/121558240_972714016543857_6033716185027836421_o.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[1.]]


In [10]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/F/121638990_1012059519260274_2330212418373086362_o.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[0.]]


In [11]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/F/121711378_1739798592838691_2434137178508018006_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))


[[0.]]


In [12]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/M/121160480_3678313258869851_5818106878740531607_o.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[1.]]


In [13]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/M/121623008_2688765031372476_1265210388791973144_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[1.]]


In [14]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/M/121558243_3459617724076248_4826110803681428223_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))

[[1.]]


In [15]:
from tensorflow.keras.preprocessing import image
# testing the model

test_image = image.load_img(test_dir + '/M/121218785_2888174644739280_5191976153731649311_n.jpg', target_size = (img_height, img_width))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(np.round(result))


[[1.]]
